In [59]:
import sys
import pandas as pd

sys.path.append('./EDA')
sys.path.append('./Modeling')

from EDA_STEP_1 import *
from visualization import *
from clustering import *

In [42]:
st_end_daram_df = get_daram_14_station_df()
st_end_daram_df.columns

Index(['ROUTE_ID', '노선명', '순번', 'NODE_ID', 'ARS_ID', '정류소명', 'X좌표', 'Y좌표',
       '법정동코드', '법정동_구', '법정동', 'academy_cnt', 'kindergarten_cnt', 'mart_cnt',
       'restaurant_cnt', 'school_cnt', 'university_cnt', 'subway_cnt',
       'tour_cnt', 'cafe_cnt', 'hospital_cnt', 'culture_cnt',
       'univ_hospital_cnt', 'public_office_cnt', 'employee_cnt', 'alone_ratio',
       'emp_corp_ratio', 'population_15to64', 'RIDE_SUM_6_10',
       'ALIGHT_SUM_6_10'],
      dtype='object')

In [34]:
start_station = ['111000128','113000113','120000156','120000109','105000127','122000305','123000209']
end_station =   ['111000291','118000048','119000024','120000018','105000072','122000302','123000043']
st_end_node_list = []
for st_end_node in zip(start_station,end_station):
    st_end_node_list.append(st_end_node)
st_end_node_list

[('111000128', '111000291'),
 ('113000113', '118000048'),
 ('120000156', '119000024'),
 ('120000109', '120000018'),
 ('105000127', '105000072'),
 ('122000305', '122000302'),
 ('123000209', '123000043')]

In [8]:
st_end_daram_df['X좌표']

5     127.0525194449
6     127.0703731962
20    126.9108335829
25    126.9343371251
30    126.9348325761
33    126.9173733044
35    126.9417570377
54    126.9280626243
62    126.9196773902
67    126.9420825336
71    127.1024565504
72    127.1026344423
81    127.1014595352
91    127.1467148058
Name: X좌표, dtype: object

In [45]:
import math
# 하버사인 공식:
## - 지구 곡률을 반영하여 두 지표 사이의 거리를 각도, 호를 사용하여 구하는 방식.
## latitude: 위도(36), longitude: 경도(127)

def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371  # 지구의 반지름

    # Convert latitude and longitude from degrees to radians
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    # Haversine formula
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    a = math.sin(dlat / 2) ** 2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c

    return distance


In [47]:
st_end_daram_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14 entries, 5 to 91
Data columns (total 30 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ROUTE_ID           14 non-null     object 
 1   노선명                14 non-null     object 
 2   순번                 14 non-null     object 
 3   NODE_ID            14 non-null     object 
 4   ARS_ID             14 non-null     object 
 5   정류소명               14 non-null     object 
 6   X좌표                14 non-null     object 
 7   Y좌표                14 non-null     object 
 8   법정동코드              14 non-null     object 
 9   법정동_구              14 non-null     object 
 10  법정동                14 non-null     object 
 11  academy_cnt        14 non-null     int32  
 12  kindergarten_cnt   14 non-null     int32  
 13  mart_cnt           14 non-null     int32  
 14  restaurant_cnt     14 non-null     int32  
 15  school_cnt         14 non-null     int32  
 16  university_cnt     14 non-nu

In [49]:
df_node_distance = pd.DataFrame(columns=['BUS_NUM','Radius'])
for bus in st_end_node_list:
    lon1 = float(st_end_daram_df['X좌표'][st_end_daram_df['NODE_ID']== bus[0]].iloc[0])
    lat1 = float(st_end_daram_df['Y좌표'][st_end_daram_df['NODE_ID']== bus[0]].iloc[0])
    lon2 = float(st_end_daram_df['X좌표'][st_end_daram_df['NODE_ID']== bus[1]].iloc[0])
    lat2 = float(st_end_daram_df['Y좌표'][st_end_daram_df['NODE_ID']== bus[1]].iloc[0])
    bus_num = st_end_daram_df['노선명'][st_end_daram_df['NODE_ID']==bus[0]].iloc[0]
    
    radius = haversine_distance(lat1, lon1, lat2, lon2)
    new_data = {'BUS_NUM':bus_num,'Radius':radius}
    df_node_distance = df_node_distance.append(new_data, ignore_index=True)


    
    

In [52]:
df_node_distance

,BUS_NUM,Radius
0,8771,2.253933
1,8761,3.270516
2,8551,3.377080
3,8552,2.189844
4,8221,2.017190
5,8441,2.383591
6,8331,4.338116


---

# 1. 기존 다람쥐버스와 같은 구, 같은동 제거.

In [146]:
gmm_daram_df = pd.read_csv('./Data/csv/gmm_daram.csv', encoding='UTF-8')
daram_df = get_daram_95station_df()


In [147]:
gmm_daram_df

,Unnamed: 0,pca1,pca2,pca3,pca4,pca5,pca6,gmm_cluster,index,X좌표,Y좌표,NODE_ID
0,1119,-0.617303,-1.459993,0.389597,3.754457,-1.562935,0.086068,0,8691,126.918884,37.521891,118000076
1,1101,0.378930,-1.079291,-0.046722,3.408468,-1.211176,-0.692579,0,9019,126.930759,37.525932,118000613
2,1164,-0.069205,1.084566,1.101227,2.018559,-1.579968,-0.601973,0,8433,126.893399,37.476205,117900029
3,1096,1.309512,-1.127871,1.382548,3.502019,-0.978033,0.237145,0,8666,126.924904,37.530782,118000051
4,1094,2.508437,-0.587880,0.831179,3.652004,-0.739060,0.164253,0,8799,126.920853,37.527967,118000196
...,...,...,...,...,...,...,...,...,...,...,...,...
1310,693,5.035301,-1.272405,3.190015,-3.294807,-0.199899,-2.680025,10,2806,127.000648,37.593218,107000140
1311,1009,1.397033,-1.372169,0.040843,0.195963,0.336420,0.366396,10,9075,126.897552,37.518672,118900051
1312,695,5.198289,-1.243730,4.051635,-2.473887,0.027647,-0.792429,10,2803,126.995944,37.593745,107000137
1313,684,4.785935,-1.268199,3.099309,-3.186075,-0.157146,-2.647045,10,2807,127.001531,37.593272,107000141


In [148]:
# 다람쥐버스 정류장 95개
daram_df = get_daram_95station_df() # 95개 다람쥐 버스 정류장
# # reset_daram_df = daram_df.reset_index() # daram_df의 index reset
# # similarity_reset_daram_df = reset_daram_df.loc[:,'academy_cnt':'population_15to64'] # 95개 df에서 숫자데이터만 추출

# # 다람쥐 버스 아닌 정류장 12453개
# reset_original_bus_df = get_not_daram_station().reset_index() 
# sim_reset_original_bus_df=reset_original_bus_df.loc[:,'academy_cnt':'population_15to64'] # 숫자 데이터만


# # # 다람쥐버스 기,종점만
# # st_end_daram_df=get_daram_14_station_df()
# # sim_st_end_daram_df=st_end_daram_df.loc[:,'academy_cnt':'population_15to64'] # 숫자 데이터만


# # # df_similar = 코사인 유사도 바탕으로 뽑힌 1400개 정류장
# # cosine_daram_df = get_cosine_similarity(sim_st_end_daram_df,sim_reset_original_bus_df,100)
# # cosine_daram_df = cosine_daram_df.drop_duplicates(subset='NODE_ID')
# # cosine_daram_df.shape

In [149]:
# 다람쥐 버스 아닌 정류장 12453개
reset_original_bus_df = get_not_daram_station()
reset_original_bus_df.columns

Index(['NODE_ID', '정류소명', 'X좌표', 'Y좌표', '법정동코드', '법정동_구', '법정동', 'academy_cnt',
       'kindergarten_cnt', 'mart_cnt', 'restaurant_cnt', 'school_cnt',
       'university_cnt', 'subway_cnt', 'tour_cnt', 'cafe_cnt', 'hospital_cnt',
       'culture_cnt', 'univ_hospital_cnt', 'public_office_cnt', 'employee_cnt',
       'alone_ratio', 'emp_corp_ratio', 'population_15to64'],
      dtype='object')

In [150]:
gmm_daram_df['NODE_ID'] = gmm_daram_df['NODE_ID'].astype('str')

In [151]:
gmm_daram_result_df = pd.merge(gmm_daram_df,reset_original_bus_df, left_on='NODE_ID' ,right_on='NODE_ID',how='left')

In [152]:
gmm_daram_result_df.columns

Index(['Unnamed: 0', 'pca1', 'pca2', 'pca3', 'pca4', 'pca5', 'pca6',
       'gmm_cluster', 'index', 'X좌표_x', 'Y좌표_x', 'NODE_ID', '정류소명', 'X좌표_y',
       'Y좌표_y', '법정동코드', '법정동_구', '법정동', 'academy_cnt', 'kindergarten_cnt',
       'mart_cnt', 'restaurant_cnt', 'school_cnt', 'university_cnt',
       'subway_cnt', 'tour_cnt', 'cafe_cnt', 'hospital_cnt', 'culture_cnt',
       'univ_hospital_cnt', 'public_office_cnt', 'employee_cnt', 'alone_ratio',
       'emp_corp_ratio', 'population_15to64'],
      dtype='object')

In [154]:
gmm_daram_result_df['구-동']=gmm_daram_result_df['법정동_구']+gmm_daram_result_df['법정동']
# 기존 다람쥐 버스 (95개)가 있는 동을 리스트로 뽑기 
daram_95_gu_list = daram_df['법정동_구'].drop_duplicates().tolist()
daram_95_gu_dong_list = daram_df['법정동'].drop_duplicates().tolist()
daram_95_gu_dong_list

['장안동',
 '답십리동',
 '신사동',
 '응암동',
 '구산동',
 '역촌동',
 '창전동',
 '노고산동',
 '여의도동',
 '노량진동',
 '상도동',
 '봉천동',
 '신림동',
 '수서동',
 '세곡동',
 '자곡동',
 '신천동',
 '방이동',
 '오금동',
 '마천동']

In [155]:
# 다람쥐 버스가 있던 동 제거
drop_cosine_daram_df = gmm_daram_result_df[~gmm_daram_result_df['구-동'].isin(daram_95_gu_dong_list)]
# 다람쥐 버스가 있던 구 제거
drop_gu_cosine_daram_df = drop_cosine_daram_df[~drop_cosine_daram_df['법정동_구'].isin(daram_95_gu_list)]

In [156]:
drop_gu_cosine_daram_df.columns

Index(['Unnamed: 0', 'pca1', 'pca2', 'pca3', 'pca4', 'pca5', 'pca6',
       'gmm_cluster', 'index', 'X좌표_x', 'Y좌표_x', 'NODE_ID', '정류소명', 'X좌표_y',
       'Y좌표_y', '법정동코드', '법정동_구', '법정동', 'academy_cnt', 'kindergarten_cnt',
       'mart_cnt', 'restaurant_cnt', 'school_cnt', 'university_cnt',
       'subway_cnt', 'tour_cnt', 'cafe_cnt', 'hospital_cnt', 'culture_cnt',
       'univ_hospital_cnt', 'public_office_cnt', 'employee_cnt', 'alone_ratio',
       'emp_corp_ratio', 'population_15to64', '구-동'],
      dtype='object')

In [157]:
get_clustering_folium(drop_gu_cosine_daram_df, 'X좌표_y', 'Y좌표_y', label_column='gmm_cluster')

In [134]:
gmm_daram_result_df['구-동']= gmm_daram_result_df['법정동_구']+gmm_daram_result_df['법정동']
gmm_daram_result_df['구-동'] = gmm_daram_result_df['구-동'].str.strip()

In [135]:
daram_df['구-동'] = daram_df['법정동_구']+daram_df['법정동']
daram_df['구-동'] = daram_df['구-동'].str.strip()

daram_95_gu_dong_list = daram_df['구-동'].drop_duplicates().tolist()
daram_95_gu_list = daram_df['법정동_구'].drop_duplicates().tolist()

In [136]:
daram_95_gu_list

['동대문구', '은평구', '마포구', '영등포구', '동작구', '관악구', '강남구', '송파구']

In [139]:
daram_95_gu_dong_list

['동대문구장안동',
 '동대문구답십리동',
 '은평구신사동',
 '은평구응암동',
 '은평구구산동',
 '은평구역촌동',
 '마포구창전동',
 '마포구노고산동',
 '영등포구여의도동',
 '동작구노량진동',
 '동작구상도동',
 '관악구봉천동',
 '관악구신림동',
 '강남구수서동',
 '강남구세곡동',
 '강남구자곡동',
 '송파구신천동',
 '송파구방이동',
 '송파구오금동',
 '송파구마천동']

In [140]:
# 다람쥐 버스가 있던 동 제거
drop_cosine_daram_df = gmm_daram_result_df[~gmm_daram_result_df['구-동'].isin(daram_95_gu_dong_list)]
drop_cosine_daram_df.shape

(572, 36)

In [119]:
drop_cosine_daram_df.columns

Index(['Unnamed: 0', 'pca1', 'pca2', 'pca3', 'pca4', 'pca5', 'pca6',
       'gmm_cluster', 'index', 'X좌표_x', 'Y좌표_x', 'NODE_ID', '정류소명', 'X좌표_y',
       'Y좌표_y', '법정동코드', '법정동_구', '법정동', 'academy_cnt', 'kindergarten_cnt',
       'mart_cnt', 'restaurant_cnt', 'school_cnt', 'university_cnt',
       'subway_cnt', 'tour_cnt', 'cafe_cnt', 'hospital_cnt', 'culture_cnt',
       'univ_hospital_cnt', 'public_office_cnt', 'employee_cnt', 'alone_ratio',
       'emp_corp_ratio', 'population_15to64', '구-동'],
      dtype='object')

In [141]:
get_clustering_folium(drop_cosine_daram_df, 'X좌표_y', 'Y좌표_y', label_column='gmm_cluster')

In [142]:
# 다람쥐 버스가 있던 구 제거
drop_cosine_daram_df = drop_cosine_daram_df[~drop_cosine_daram_df['법정동_구'].isin(daram_95_gu_list)]
drop_cosine_daram_df.shape

(322, 36)

In [143]:
get_clustering_folium(drop_cosine_daram_df, 'X좌표_y', 'Y좌표_y', label_column='gmm_cluster')

In [124]:
drop_cosine_daram_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 322 entries, 2 to 1313
Data columns (total 36 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         322 non-null    int64  
 1   pca1               322 non-null    float64
 2   pca2               322 non-null    float64
 3   pca3               322 non-null    float64
 4   pca4               322 non-null    float64
 5   pca5               322 non-null    float64
 6   pca6               322 non-null    float64
 7   gmm_cluster        322 non-null    int64  
 8   index              322 non-null    int64  
 9   X좌표_x              322 non-null    float64
 10  Y좌표_x              322 non-null    float64
 11  NODE_ID            322 non-null    object 
 12  정류소명               322 non-null    object 
 13  X좌표_y              322 non-null    object 
 14  Y좌표_y              322 non-null    object 
 15  법정동코드              322 non-null    object 
 16  법정동_구              322 no

In [125]:
get_clustering_folium(drop_cosine_daram_df, 'X좌표_y', 'Y좌표_y', label_column='gmm_cluster')